In [4]:
import pickle
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

In [2]:
df = pd.read_csv(r'C:\Users\Nitin Flavier\Desktop\Data Nexus\Data Science\ML_BootCamp\ML_Projects\Churn_Modeling_ANN\datasets\transformed_churn_data.csv')
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [5]:
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.models import Sequential 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [6]:
X_train.shape

(6700, 12)

In [8]:
model = Sequential([
    Dense(64,activation=LeakyReLU(alpha=0.01),input_shape=(X_train.shape[1],)),
    Dense(32,activation=LeakyReLU(alpha=0.01)),
    Dense(1)
])

# compile the model
loss = tf.keras.losses.MeanAbsoluteError()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# compile the model 
model.compile(optimizer=opt,loss=loss,metrics=['mae'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
import datetime
# Setting up Tensorboard for training 
log_dir = r"..\regression_logs\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflowCallback = TensorBoard(log_dir=log_dir,histogram_freq=1)
# waits for 5 epochs if the loss does change much it will stop
earlyStoppingCallback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [11]:
history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[earlyStoppingCallback,tensorflowCallback]
)

Epoch 1/100


210/210 [==============================] - 2s 3ms/step - loss: 88700.4375 - mae: 88700.4375 - val_loss: 56022.0117 - val_mae: 56022.0117
Epoch 2/100
210/210 [==============================] - 0s 2ms/step - loss: 51447.9766 - mae: 51447.9766 - val_loss: 49989.7812 - val_mae: 49989.7812
Epoch 3/100
210/210 [==============================] - 0s 2ms/step - loss: 50252.2930 - mae: 50252.2930 - val_loss: 49886.7812 - val_mae: 49886.7812
Epoch 4/100
210/210 [==============================] - 0s 2ms/step - loss: 50076.8906 - mae: 50076.8906 - val_loss: 49660.8438 - val_mae: 49660.8438
Epoch 5/100
210/210 [==============================] - 0s 2ms/step - loss: 49926.5859 - mae: 49926.5859 - val_loss: 49628.2773 - val_mae: 49628.2773
Epoch 6/100
210/210 [==============================] - 0s 2ms/step - loss: 49861.5742 - mae: 49861.5742 - val_loss: 49589.1719 - val_mae: 49589.1719
Epoch 7/100
210/210 [==============================] - 0s 2ms/step - loss: 49782.4531 - mae: 49782.4531 

In [12]:
%load_ext tensorboard

In [17]:
%tensorboard --logdir ../regression_logs/fit


Reusing TensorBoard on port 6007 (pid 3088), started 0:00:04 ago. (Use '!kill 3088' to kill it.)

In [18]:
test_loss,test_mae = model.evaluate(X_test,y_test)

print(f"Loss: {test_loss}")
print(f"MAE: {test_mae}")

104/104 [==============================] - 0s 790us/step - loss: 49589.1719 - mae: 49589.1719
Loss: 49589.171875
MAE: 49589.171875
